# For Sample
[Convolutional Network Visualizations & Deep Dream](https://www.kaggle.com/code/carloalbertobarbano/convolutional-network-visualizations-deep-dream/notebook)

In [22]:
import numpy as np
import torch
from torch.autograd import Variable
from torch.optim import SGD
from torchvision import models, transforms
import PIL
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import scipy.ndimage as ndimage

%matplotlib inline

import scipy.ndimage as nd
import PIL.Image
from IPython.display import clear_output, Image, display
from io import BytesIO

In [28]:
MEAN_DEFINE = [0.485, 0.456, 0.406]
STD_DEFINE = [0.229, 0.224, 0.225]
IMAGE_RESIZE = (224, 224)

# np.uint8(np.clip(image, 0, 255))
* 0 ~ 255 の値に収め、uint8のnp型にキャストする

# バイト列を格納するBufferを用意する
* buffer = BytesIO()

In [24]:
def show_array(image, format='jpg'):
    clip_np_image = np.uint8(np.clip(image, 0, 255))
    buffer = BytesIO()
    
    PIL.Image.fromarray(clip_np_image).save(buffer, format)
    display(Image(data=buffer.getvalue()))

# reshapeは配列を形状変換する
* mean(平均)
    * [0.485 0.456 0.406] -> [[[0.485 0.456 0.406]]]
* std(標準偏差)
    * [0.229, 0.224, 0.225] -> [[[0.229, 0.224, 0.225]]]
* inp(インプット)

In [25]:
def show_tensor(image):
    maen = np.array(MEAN_DEFINE).reshape([1, 1, 3])
    std = np.array(STD_DEFINE).reshape([1, 1, 3])

    # インプット
    inp = image[0, :, :, :]
    inp = inp.transpose(1, 2, 0)
    inp = std * inp + mean
    inp *= 255

    showarray(inp)
    clear_output(wait=True)

In [26]:
def plot_imaages(image, title=None):
    plt.figure(figsize=(30, 20))
    
    for i in range(len(images)):
        plt.subplot(10 / 5 + 1, 5, i + 1)
        plt.axis('off')
        if titles is not None:
            plt.title(titles[i])
        plt.imshow(images[i])
        
    plt.pause(0.001)

# Resize
* リサイズを行うTransform
# ToTensor
* PIL Image をテンソルに変換する Transform
* 値の範囲は [0, 1] の float にスケールされる
* 形状は (C, H, W) になる
# Normalize
* 正規化を行う Transform 
* n 個のチャンネルごとの平均 (m1,m2,⋯ ,mn),及び標準偏差 (s1,s2,⋯ ,sn) が与えられたとき、チャンネルごとに次のように標準化を行います。
$$ 
    output_c = \frac{input_c – m_c}s_c
$$

In [30]:
normalise = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(MEAN_DEFINE, STD_DEFINE)
])

normalise_resize = transforms.Compose([
    transforms.Resize(IMAGE_RESIZE),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_DEFINE, STD_DEFINE)
])

# np.full
* 任意の値で全要素を初期化したndarrayを生成する
* np.full(n, m, z) # n行 * m列 z次元

# numpy.random.uniform(low, high, size)
* 任意の範囲の連続一様分布から浮動小数点数の乱数を生成する

# torch.unsqueeze
* 指定した位置にサイズ1の次元を挿入する

In [33]:
def initialize_image(size=(400, 400, 3)):
    initalize_image = PIL.Image.fromarray(np.uint8(np.full(size, 150)))
    random_create_image = PIL.Image.fromarray(np.uint8(np.random.uniform(150, 180, size)))
    image_tensor = normalise(random_create_image).unsqueeze(0)
    image_np = img_tensor.numpy()
    return random_create_image, image_tensor, image_np

# PIL.Image.ANTIALIAS
* アンチエイリアスで写真をキレイに縮小


In [36]:
def load_path(path, resize=False, size=None):
    image = PIL.Image.open(path)
    
    if size is not None:
        image.thumbnailmb(size, PIL.Image.ANTIALIAS)
        
    if resize:
        image_tensor = normalise_resize(image).unsqueeze(0)
    else:
        image_tensor = normalise(image).unsquee(0)
    image_np = image_tensor.numpy()
    
    return image, image_tensor, image_np

In [37]:
def tensor_to_image(tensor):
    to_np = tensor.numpy()
    mean = np.array(MEAN_DEFINE).reshape([1, 1, 3])
    std = np.array(STD_DEFINE).reshape([1, 1, 3])
    inp = to_np[0, :, :, :]
    inp = inp.transpose(1, 2, 0)
    inp = std * inp + mean
    inp *= 255
    inp = np.uint8(np.clip(inp, 0, 255))
    return PIL.Image.fromarray(inp)